# AI/ML Assessment 2

## Imports

In [2]:
# To retrieve requirements to run this notebook, run: !pip freexe > requirements.txt, for a user to install the requirements, run: !pip install -r requirements.txt
import pandas as pd
import seaborn as sns
import plotly.express as px